In [1]:
version = 102

In [2]:
import pandas as pd
import numpy as np

## Loading Data

In [3]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v19.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v19.csv")
submission_set = pd.read_csv("./data/final_submission_set.csv")

## Predictive Model Learning

In [4]:
cat_names = ['callsign',
            'adep', 
            'ades', 
            'aircraft_type', 
            'wtc', 
            'airline',
            'offblock_hour',
            'offblock_minute', 
            'offblock_day_of_week',
            'offblock_weekday_name',
            'offblock_month',
            'offblock_week_of_year', 
            'offblock_season', 
            'arrival_hour',
            'arrival_minute',
            'arrival_season',
            'arrival_weekday_name',
            'is_offblock_weekend',
            'is_offblock_rush_hour',
            'flight_duration_category',                       
            'adep_region', 
            'ades_region', 
            'same_country_flight',
            'same_region_flight',                        
            'flight_direction',
            'is_intercontinental',
            'Manufacturer',
            'Model_FAA',
            'Physical_Class_Engine',
            'FAA_Weight',
            'adep_geo_cluster',
            'ades_geo_cluster']

In [5]:
dataset = pd.concat([challenge_set_updated, submission_set_updated], axis=0)

In [6]:
pd.set_option('display.max_rows', None)

print(dataset.isnull().mean().sort_values(ascending=False))

pd.reset_option('display.max_rows')

vlof_tas                            0.406441
sqrd_tas_1                          0.406441
tas_1                               0.406441
specific_energy_1                   0.406441
sqrd_vlof_tas                       0.406441
vlof_groundspeed                    0.406441
tas_2                               0.372550
specific_energy_2                   0.372550
sqrd_tas_2                          0.372550
sqrd_tas_3                          0.359277
specific_energy_3                   0.359277
tas_3                               0.359277
sqrd_tas_10                         0.356763
specific_energy_10                  0.356763
tas_10                              0.356763
sqrd_tas_9                          0.353705
tas_9                               0.353705
specific_energy_9                   0.353705
tas_4                               0.353320
specific_energy_4                   0.353320
sqrd_tas_4                          0.353320
tas_8                               0.351376
sqrd_tas_8

In [7]:
# Dropping columns with too many NaNs
threshold = 0.4
df = dataset.dropna(thresh=int((1 - threshold) * len(dataset)), axis=1)

In [8]:
# Imputation of NaNs
columns_with_nan = dataset.isna().any()
for col in dataset.columns[columns_with_nan]:
    dataset.loc[:, col] = dataset.fillna(dataset[col].median())

In [9]:
df = dataset.iloc[0:challenge_set_updated.shape[0], :]

In [10]:
X = df.drop('tow', axis=1)
y = df.tow

In [11]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Create correlation matrix
# corr_matrix = X.corr(numeric_only=True).abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# # Find features with correlation greater than 0.98
# to_drop = [column for column in upper.columns if any(upper[column] > 0.98)]
# print(to_drop)

# # Plotting the correlation matrix
# plt.figure(figsize=(12, 8))
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
# plt.title('Correlation Matrix of Challenge Set')
# plt.show()

# # # Drop features 
# # X.drop(to_drop, axis=1, inplace=True)

In [12]:
eliminated_features = ['groundspeed_airspeed_ratio_ENR', 'temperature_9', 'wind_distance_flown_distance_ENR', 'average_humidity_DEP_40', 'vertical_rate_bins_ARR', 
        'groundspeed_flown_distance_ARR', 'arrival_quarter', 'offblock_year', 'arrival_year', 'offblock_to_arrival_day_diff', 'altitude_9', 'tas_1', 
        'is_arrival_weekend', 'adep_height_6', 'sqrd_vlof_tas', 'average_airspeed_ARR_100', 'adep_height_7', 'wind_distance_ARR_100', 'altitude_4', 
        'adep_height_1', 'groundspeed_airspeed_ratio_ARR', 'tas_8', 'specific_energy_4', 'temperature_bins_DEP', 'temperature_6', 'humidity_bins_DEP', 
        'altitude_5', 'adep_height_5', 'sqrd_tas_8', 'sqrd_tas_7', 'specific_energy_7', 'specific_energy_1', 'adep_height_4', 'sqrd_tas_6', 'tas_2', 
        'sqrd_tas_5', 'specific_energy_3', 'altitude_8', 'specific_energy_6', 'adep_height_8', 'vertical_rate_airspeed_ARR', 'altitude_2', 'sqrd_tas_1', 
        'sqrd_tas_3', 'specific_energy_8', 'sqrd_tas_9', 'temperature_8', 'groundspeed_airspeed_ratio_DEP', 'sqrd_tas_4', 'altitude_6', 
        'specific_energy_5', 'humidity_temperature_DEP', 'adep_height_2', 'altitude_7', 'adep_height_3', 'temperature_1', 'specific_energy_2', 
        'temperature_5', 'wind_distance_flown_distance_ARR', 'arrival_month', 'temperature_4', 'groundspeed_ARR_100', 'tas_4', 'arrival_minute', 
        'adep_height_9', 'altitude_groundspeed_ARR', 'altitude_3', 'temperature_7', 'airspeed_specific_energy_ENR', 'altitude_10', 'sqrd_tas_10', 
        'humidity_bins_ARR', 'specific_energy_9', 'sqrd_tas_2', 'temperature_2', 'tas_10', 'average_humidity_ENR', 'offblock_quarter', 
        'airspeed_specific_energy_DEP', 'wind_distance_flown_distance_DEP', 'tas_6', 'flown_distance_ARR_100', 'vertical_rate_airspeed_ratio_ARR', 
        'average_humidity_ARR_100', 'specific_energy_10', 'first_adep_height', 'tas_3', 'temperature_3', 'track_variation_ARR_100', 
        'is_offblock_rush_hour', 'average_temperature_ENR', 'is_arrival_rush_hour', 'average_altitude_ARR_100', 'specific_energy_ENR', 
        'groundspeed_ENR', 'is_offblock_weekend', 'Num_Engines', 'temperature_bins_ARR', 'average_temperature_ARR_100', 'kpi17_time', 
        'average_airspeed_DEP_40', 'wind_distance_ENR', 'offblock_minute', 'groundspeed_10NM', 'average_vertical_rate_ARR_100', 'vlof_tas', 
        'humidity_temperature_ARR']

In [13]:
X.drop(eliminated_features, axis=1, inplace=True)

In [ ]:
from catboost import CatBoostRegressor, Pool, metrics
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'learning_rate': 0.01, 
    'reg_lambda': 0.05357182104973179, 
    'random_strength': 20.10951792232919, 
    'depth': 9, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 12
}

selected_cat_names = [x for x in cat_names if x in X.columns]

train_pool = Pool(X_train, y_train, cat_features=selected_cat_names)
val_pool = Pool(X_val, y_val, cat_features=selected_cat_names)

model = CatBoostRegressor(
    iterations=100000,
    objective=metrics.RMSE(),
    eval_metric=metrics.RMSE(),
    random_seed=42,
    verbose=False,
    task_type='GPU',
    use_best_model=True,
    od_type='Iter',
    od_wait=50,
    **params,
)

In [ ]:
model.fit(
    train_pool, eval_set=val_pool,
    plot=True
)

In [ ]:
model.save_model('catboost_train_v%d.cbm' % version, 'cbm')

In [ ]:
y_pred = model.predict(X_val)
pd.DataFrame(data={'tow': y_pred}).to_csv('catboost_val_v%d.csv' % version, index=False)

In [ ]:
def classify_aircraft(row):
    if row['Physical_Class_Engine'] == 'Turboprop' and row['wtc'] == 'M':
        return 'Medium Turbo Prop'
    elif row['Physical_Class_Engine'] == 'Jet' and row['wtc'] == 'M':
        return 'Medium Jet'
    elif row['Physical_Class_Engine'] == 'Jet' and row['wtc'] == 'H':
        return 'Heavy Jet'
    else:
        return None  # If no classification applies, return None

In [ ]:
from sklearn.metrics import root_mean_squared_error

X_val_groups = X_val.copy()

X_val_groups['Aircraft_Class'] = X_val.apply(classify_aircraft, axis=1)

X_val_groups = X_val_groups.reset_index()

indices_m_prop = X_val_groups[X_val_groups['Aircraft_Class'] == 'Medium Turbo Prop'].index
indices_m_jet = X_val_groups[X_val_groups['Aircraft_Class'] == 'Medium Jet'].index
indices_h_jet = X_val_groups[X_val_groups['Aircraft_Class'] == 'Heavy Jet'].index

print('RMSE Medium Turbo Prop:', root_mean_squared_error(y_val.iloc[indices_m_prop], y_pred.iloc[indices_m_prop]))
print('RMSE Medium Jet:', root_mean_squared_error(y_val.iloc[indices_m_jet], y_pred.iloc[indices_m_jet]))
print('RMSE Heavy Jet:', root_mean_squared_error(y_val.iloc[indices_h_jet], y_pred.iloc[indices_h_jet]))

In [ ]:
best_iteration = model.get_best_iteration()
model = CatBoostRegressor(
    iterations=best_iteration,
    objective=metrics.RMSE(),
    random_seed=42,
    verbose=False,
    task_type='GPU',
    **params,
)
model.fit(
    X, y,
    cat_features=selected_cat_names, 
    plot=True
)

In [ ]:
model.save_model('catboost_all_v%d.cbm' % version, 'cbm')

In [ ]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

In [ ]:
import matplotlib

feature_im_df = pd.DataFrame({
    "feature": X.columns,
    "importance": feature_importances
})

feature_im_df = feature_im_df.sort_values(by="importance", ascending=False)

sns.set(rc={'text.usetex' : True, 'text.latex.preamble': '\\usepackage{libertine}'})
plt.figure(figsize=(10, 6))
plt.figure()
sns.barplot(data = feature_im_df[:20], x='importance', y='feature', palette="viridis")

plt.title("CatBoost feature importance")
plt.xlabel("Importance")
plt.ylabel("feature")
plt.tight_layout()
plt.savefig('catboost_feature_importance.pdf', bbox_inches='tight')
plt.show()

In [ ]:
model = CatBoostRegressor(**params).fit(train_pool)
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

## Generating Submission

In [ ]:
df_test = dataset.iloc[challenge_set_updated.shape[0]:, :]
df_test.head()

In [ ]:
X_test = df_test.drop('tow', axis=1)

In [ ]:
X_test = X_test.loc[:, X_test.columns.isin(X.columns)]

In [ ]:
y_pred = model.predict(X_test)
y_test = y_pred
print(y_test)

In [ ]:
dft0 = pd.read_csv('./data/final_submission_set.csv')
dft0['tow'] = y_test
dft0[['flight_id', 'tow']].to_csv('catboost.csv', index=False)

In [ ]:
print((df_test['aircraft_type'] == submission_set_updated['aircraft_type']).mean())

In [ ]:
submission_dataset = df_test
submission_dataset['tow'] = y_pred
pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')
print((submission_dataset['tow'] > submission_dataset['MTOW_kg']).sum())
mask = (submission_dataset['tow'] > submission_dataset['MTOW_kg'])
submission_dataset.loc[mask, 'tow'] = submission_dataset.loc[mask, 'MTOW_kg']
print((submission_dataset['tow'] > submission_dataset['MTOW_kg']).sum())

In [ ]:
dft0 = pd.read_csv('./data/final_submission_set.csv')
dft0['tow'] = submission_dataset['tow']
dft0[['flight_id', 'tow']].to_csv('catboost_submission_v%.csv' % version, index=False)